## 第一個大型專案 - 專業的你！

### 以及，工具使用。

### 但首先：介紹 Pushover

Pushover 是一個用於向您的手機發送推送通知的巧妙工具。

它的設置和安裝非常簡單！

只需造訪 https://pushover.net/ 並點擊右上角的「Login or Signup」來註冊一個免費帳戶，並創建您的 API 金鑰。

註冊完成後，在主畫面上點擊「Create an Application/API Token」，給它取一個名稱（例如 Agents），然後點擊「Create Application」。

接著在您的 `.env` 文件中新增兩行：

PUSHOVER_USER=_填入您 Pushover 主畫面右上角的金鑰，可能以 u 開頭_  
PUSHOVER_TOKEN=_填入您新創建的名為 Agents（或其他名稱）的應用程式中的金鑰，可能以 a 開頭_

記得保存您的 `.env` 文件，並在保存後執行 `load_dotenv(override=True)`，以設置您的環境變數。

最後，點擊「Add Phone, Tablet or Desktop」來安裝到您的手機上。

In [ ]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("HEY!!")

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
# JSONB是一種用於描述 tool 的 JSON 格式，這些 tool 可以在 OpenAI 的 API 中使用。
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [ ]:
# 這個函式可以接收一個工具呼叫的列表，並執行它們。這是使用 IF 條件語句的版本!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name  # 從工具呼叫中取得工具名稱
        arguments = json.loads(tool_call.function.arguments)  # 將工具呼叫的參數轉換為字典
        print(f"Tool called: {tool_name}", flush=True)  # 印出被呼叫的工具名稱

        # 這是主要的 IF 條件語句!!!

        if tool_name == "record_user_details":  # 如果工具名稱是 record_user_details
            result = record_user_details(**arguments)  # 呼叫 record_user_details 函式並傳入參數
        elif tool_name == "record_unknown_question":  # 如果工具名稱是 record_unknown_question
            result = record_unknown_question(**arguments)  # 呼叫 record_unknown_question 函式並傳入參數

        # 將結果加入結果列表
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results  # 返回所有工具呼叫的結果

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

In [ ]:
# 這是一種更優雅的方式，避免使用 IF 條件語句。

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name  # 從工具呼叫中取得工具名稱
        arguments = json.loads(tool_call.function.arguments)  # 將工具呼叫的參數轉換為字典
        print(f"Tool called: {tool_name}", flush=True)  # 印出被呼叫的工具名稱
        tool = globals().get(tool_name)  # 從全域變數中取得對應的工具函式
        result = tool(**arguments) if tool else {}  # 如果工具存在，執行工具函式，否則返回空字典
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})  # 將結果加入結果列表
    return results  # 返回所有工具呼叫的結果

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [ ]:
system_prompt = f"您正在扮演 {name} 的角色。您正在 {name} 的網站上回答問題，\
特別是與 {name} 的職業、生涯背景、技能和經驗相關的問題。\
您的責任是盡可能忠實地代表 {name} 與網站上的訪客互動。\
您將獲得一份 {name} 的背景摘要和 LinkedIn 簡介，您可以使用這些資訊來回答問題。\
請保持專業且具有吸引力，就像在與潛在客戶或未來雇主交談一樣，他們可能偶然發現了這個網站。\
如果您不知道任何問題的答案，請使用您的 record_unknown_question 工具記錄您無法回答的問題，即使這些問題是關於一些瑣碎或與職業無關的事情。\
如果用戶正在進行討論，請嘗試引導他們通過電子郵件聯繫；詢問他們的電子郵件，並使用您的 record_user_details 工具記錄下來。"

system_prompt += f"\n\n## 摘要:\n{summary}\n\n## LinkedIn 簡介:\n{linkedin}\n\n"
system_prompt += f"基於這些背景資訊，請與用戶聊天，始終以 {name} 的角色進行對話。"


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

# 現在進行部署

此程式碼位於 `app.py`

我們將部署到 HuggingFace Spaces。

在開始之前：記得更新 "me" 目錄中的檔案 - 您的 LinkedIn 簡介和 summary.txt - 讓它能夠描述您！同時在 `app.py` 中更改 `self.name = "Ed Donner"`。

另外，檢查 1_foundations 目錄中是否有 README 檔案。如果有，請刪除它。部署過程會在此目錄中為您創建一個新的 README 檔案。

1. 造訪 https://huggingface.co 並建立一個帳戶  
2. 從右上角的頭像選單中選擇 Access Tokens。選擇 "Create New Token"。給它 WRITE 權限 - 它需要具有 WRITE 權限！記下您的新金鑰。  
3. 在終端機中執行：`uv tool install 'huggingface_hub[cli]'` 來安裝 HuggingFace 工具，然後執行 `hf auth login` 使用您的金鑰在命令列中登入。之後，執行 `hf auth whoami` 檢查您是否已登入  
4. 將您的新金鑰新增到您的 .env 檔案中：`HF_TOKEN=hf_xxx` 以供未來使用  
5. 從 1_foundations 資料夾中輸入：`uv run gradio deploy`  
6. 按照指示操作：命名為 "career_conversation"，指定 app.py，選擇 cpu-basic 作為硬體，選擇需要提供密鑰，提供您的 openai api key、pushover user 和 token，並選擇 "no" 不使用 github actions。  

感謝 Robert、James、Martins、Andras 和 Priya 提供這些提示。  
請閱讀接下來的兩個部分 - 如何更改您的密鑰，以及如何重新部署您的 Space（您可能需要刪除在此 1_foundations 目錄中創建的 README.md）。

#### 關於這些密鑰的更多資訊：

如果您對這些密鑰的操作感到困惑：它只是要求您輸入每個密鑰的名稱和值 -- 因此您需要輸入：  
`OPENAI_API_KEY`  
接著輸入：  
`sk-proj-...`  

如果您不想以這種方式設置密鑰，或者出現問題，也沒關係 - 您可以稍後更改您的密鑰：  
1. 登入 HuggingFace 網站  
2. 通過右上角的頭像選單進入您的個人資料頁面  
3. 選擇您部署的 Space  
4. 點擊右上角的設定齒輪  
5. 您可以向下滾動更改您的密鑰（變數和密鑰部分）、刪除 Space 等。

#### 現在您應該已經部署完成！

如果您想完全替換所有內容並重新開始使用您的密鑰，您可能需要刪除在此 1_foundations 資料夾中創建的 README.md。

這是我的範例：https://huggingface.co/spaces/ed-donner/Career_Conversation

我剛收到一個推送通知，有位學生問我如何成為他們國家的總統 😂😂

有關部署的更多資訊：

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

若要在未來刪除您的 Space：  
1. 登入 HuggingFace  
2. 從頭像選單中選擇您的個人資料  
3. 點擊該 Space 並選擇右上角的設定齒輪  
4. 向下滾動到底部的刪除部分  
5. 另外：刪除 Gradio 可能在此 1_foundations 資料夾中創建的 README 檔案（否則下次執行 gradio deploy 時不會再次詢問您問題）# And now for deployment

<!-- This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy) -->


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>